<h1 style="color:#3da1da;font-size: 300%;" id="timeshap tutorial" align="center"  >TimeSHAP Tutorial - TensorFlow - AReM dataset</h1><p>&nbsp;

<a id='top_cell'></a>

## Table of contents
1. [Data Processing](#1.-Data-Processing)
  1. [Data Loading](#1.1-Data-Loading)
  2. [Data Treatment](#1.2-Data-Treatment)
2. [Model](#2.-Model)
  1. [Model Definition](#2.1-Model-Definition)
  2. [Model Training](#2.2-Model-Training)
3. [TimeSHAP](#3.-TimeSHAP)
  1. [Local Explanations](#3.1-Local-Explanations)
  2. [Global Explanations](#3.2-Global-Explanations)
  3. [Individual Plots](#3.3-Individual-Plots)
    

# TimeSHAP

TimeSHAP is a model-agnostic, recurrent explainer that builds upon KernelSHAP and extends it to the sequential domain. 

TimeSHAP computes local event/timestamp- feature-, and cell-level attributions. 
    
Aditionally TimeSHAP also computes global event- and feature-level explanations.
    
As sequences can be arbitrarily long, TimeSHAP also implements a pruning algorithm based on Shapley Values, 
that finds a subset of consecutive, recent events that contribute the most to the decision.

---
# 1. Data-Processing
---

In [1]:
import pandas as pd
import numpy as np

np.random.seed(42)

import warnings
warnings.filterwarnings('ignore')

In [2]:
from timeshap import __version__
__version__

'1.0.4'

In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## 1.1 Data-Loading

In [4]:
import os
import re

data_directories = next(os.walk("./AReM"))[1]

all_csvs = []
for folder in data_directories:
    if folder in ['bending1', 'bending2']:
        continue
    folder_csvs = next(os.walk(f"./AReM/{folder}"))[2]
    for data_csv in folder_csvs:
        if data_csv == 'dataset8.csv' and folder == 'sitting':
            # this dataset only has 479 instances
            # it is possible to use it, but would require padding logic
            continue
        loaded_data = pd.read_csv(f"./AReM/{folder}/{data_csv}", skiprows=4)
        print(f"{folder}/{data_csv} ------ {loaded_data.shape}")
        
        csv_id = re.findall(r'\d+', data_csv)[0]
        loaded_data['id'] = csv_id
        loaded_data['all_id'] = f"{folder}_{csv_id}"
        loaded_data['activity'] = folder
        all_csvs.append(loaded_data)
all_data = pd.concat(all_csvs)
raw_model_features = ['avg_rss12', 'var_rss12', 'avg_rss13', 'var_rss13', 'avg_rss23', 'var_rss23']
all_data.columns = ['timestamp', 'avg_rss12', 'var_rss12', 'avg_rss13', 'var_rss13', 'avg_rss23', 'var_rss23', 'id', 'all_id', 'activity']

cycling/dataset1.csv ------ (480, 7)
cycling/dataset10.csv ------ (480, 7)
cycling/dataset11.csv ------ (480, 7)
cycling/dataset12.csv ------ (480, 7)
cycling/dataset13.csv ------ (480, 7)
cycling/dataset14.csv ------ (480, 7)
cycling/dataset15.csv ------ (480, 7)
cycling/dataset2.csv ------ (480, 7)
cycling/dataset3.csv ------ (480, 7)
cycling/dataset4.csv ------ (480, 7)
cycling/dataset5.csv ------ (480, 7)
cycling/dataset6.csv ------ (480, 7)
cycling/dataset7.csv ------ (480, 7)
cycling/dataset8.csv ------ (480, 7)
cycling/dataset9.csv ------ (480, 7)
lying/dataset1.csv ------ (480, 7)
lying/dataset10.csv ------ (480, 7)
lying/dataset11.csv ------ (480, 7)
lying/dataset12.csv ------ (480, 7)
lying/dataset13.csv ------ (480, 7)
lying/dataset14.csv ------ (480, 7)
lying/dataset15.csv ------ (480, 7)
lying/dataset2.csv ------ (480, 7)
lying/dataset3.csv ------ (480, 7)
lying/dataset4.csv ------ (480, 7)
lying/dataset5.csv ------ (480, 7)
lying/dataset6.csv ------ (480, 7)
lying/dataset

## 1.2 Data Treatment

### Separate in train and test

In [5]:
# choose ids to use for test
ids_for_test = np.random.choice(all_data['id'].unique(), size = 4, replace=False)

d_train =  all_data[~all_data['id'].isin(ids_for_test)]
d_test = all_data[all_data['id'].isin(ids_for_test)]

In [6]:
all_data.shape

(35520, 10)

In [7]:
###  Normalize Features

###  Normalize Features

In [7]:
class NumericalNormalizer:
    def __init__(self, fields: list):
        self.metrics = {}
        self.fields = fields

    def fit(self, df: pd.DataFrame ) -> list:
        means = df[self.fields].mean()
        std = df[self.fields].std()
        for field in self.fields:
            field_mean = means[field]
            field_stddev = std[field]
            self.metrics[field] = {'mean': field_mean, 'std': field_stddev}

    def transform(self, df: pd.DataFrame) -> pd.DataFrame:
        # Transform to zero-mean and unit variance.
        for field in self.fields:
            f_mean = self.metrics[field]['mean']
            f_stddev = self.metrics[field]['std']
            # OUTLIER CLIPPING to [avg-3*std, avg+3*avg]
            df[field] = df[field].apply(lambda x: f_mean - 3 * f_stddev if x < f_mean - 3 * f_stddev else x)
            df[field] = df[field].apply(lambda x: f_mean + 3 * f_stddev if x > f_mean + 3 * f_stddev else x)
            if f_stddev > 1e-5:
                df[f'p_{field}_normalized'] = df[field].apply(lambda x: ((x - f_mean)/f_stddev))
            else:
                df[f'p_{field}_normalized'] = df[field].apply(lambda x: x * 0)
        return df

In [8]:
#all features are numerical
normalizor = NumericalNormalizer(raw_model_features)
normalizor.fit(d_train)
d_train_normalized = normalizor.transform(d_train)
d_test_normalized = normalizor.transform(d_test)

### Features

In [9]:
model_features = [f"p_{x}_normalized" for x in raw_model_features]
time_feat = 'timestamp'
label_feat = 'activity'
sequence_id_feat = 'all_id'

plot_feats = {
    'p_avg_rss12_normalized': "Mean Chest <-> Right Ankle",
    'p_var_rss12_normalized': "STD Chest <-> Right Ankle",
    'p_avg_rss13_normalized': "Mean Chest <-> Left Ankle",
    'p_var_rss13_normalized': "STD Chest <-> Left Ankle",
    'p_avg_rss23_normalized': "Mean Right Ankle <-> Left Ankle",
    'p_var_rss23_normalized': "STD Right Ankle <-> Left Ankle",
}

### Transform the dataset from multi-label to binary classification

In [10]:
# possible activities ['cycling', 'lying', 'sitting', 'standing', 'walking']
#Select the activity to predict
chosen_activity = 'cycling'

d_train_normalized['label'] = d_train_normalized['activity'].apply(lambda x: int(x == chosen_activity))
d_test_normalized['label'] = d_test_normalized['activity'].apply(lambda x: int(x == chosen_activity))

This example notebook requires TensorFlow!

Install it if you haven't already:
```
!pip install tensorflow
```

In [11]:
def df_to_numpy(df, model_feats, label_feat, group_by_feat, timestamp_Feat):
    sequence_length = len(df[timestamp_Feat].unique())

    data_tensor = np.zeros(
        (len(df[group_by_feat].unique()), sequence_length, len(model_feats)))
    labels_tensor = np.zeros((len(df[group_by_feat].unique()), 1))

    for i, name in enumerate(df[group_by_feat].unique()):
        name_data = df[df[group_by_feat] == name]
        sorted_data = name_data.sort_values(timestamp_Feat)

        data_x = sorted_data[model_feats].values
        labels = sorted_data[label_feat].values
        assert labels.sum() == 0 or labels.sum() == len(labels)
        data_tensor[i, :, :] = data_x
        labels_tensor[i, :] = labels[0]
    return data_tensor, labels_tensor

In [12]:
X_train, y_train = df_to_numpy(d_train_normalized, model_features, 'label', sequence_id_feat, time_feat)

X_test, y_test = df_to_numpy(d_test_normalized, model_features, 'label', sequence_id_feat, time_feat)

___
# 2. Model


This example notebook requires Tensorflow!

Install it if you haven't already:
```
!pip install tensorflow
```

## 2.1 Model Definition

In [13]:
import tensorflow as tf

inputs = tf.keras.layers.Input(shape=(None, 6))
lstm1 = tf.keras.layers.LSTM(64)(inputs)
ff1 = tf.keras.layers.Dense(64, activation='relu')(lstm1)
ff2 = tf.keras.layers.Dense(1, activation='sigmoid')(ff1)
model = tf.keras.models.Model(inputs=inputs, outputs=ff2)

## 2.2 Model Training

In [14]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(0.001))

model.fit(X_train, y_train, epochs=20, batch_size=55, validation_data=(X_test, y_test))

Epoch 1/20
1/1 [==============================] - 3s 3s/step - loss: 0.6783 - val_loss: 0.6586
Epoch 2/20
1/1 [==============================] - 0s 256ms/step - loss: 0.6452 - val_loss: 0.6305
Epoch 3/20
1/1 [==============================] - 0s 329ms/step - loss: 0.6137 - val_loss: 0.6028
Epoch 4/20
1/1 [==============================] - 0s 299ms/step - loss: 0.5837 - val_loss: 0.5756
Epoch 5/20
1/1 [==============================] - 0s 350ms/step - loss: 0.5545 - val_loss: 0.5487
Epoch 6/20
1/1 [==============================] - 0s 284ms/step - loss: 0.5256 - val_loss: 0.5223
Epoch 7/20
1/1 [==============================] - 0s 299ms/step - loss: 0.4968 - val_loss: 0.4958
Epoch 8/20
1/1 [==============================] - 1s 526ms/step - loss: 0.4680 - val_loss: 0.4686
Epoch 9/20
1/1 [==============================] - 0s 306ms/step - loss: 0.4388 - val_loss: 0.4415
Epoch 10/20
1/1 [==============================] - 0s 355ms/step - loss: 0.4092 - val_loss: 0.4142
Epoch 11/20
1/1 [=====

---
# 3. TimeSHAP
---

In [15]:
import timeshap

### Model entry point

In [16]:
f = lambda x: model.predict(x)

### Baseline event

In [17]:
from timeshap.utils import calc_avg_event
average_event = calc_avg_event(d_train_normalized, numerical_feats=model_features, categorical_feats=[])

In [18]:
average_event

,p_avg_rss12_normalized,p_var_rss12_normalized,p_avg_rss13_normalized,p_var_rss13_normalized,p_avg_rss23_normalized,p_var_rss23_normalized
0,0.082851,-0.525409,0.145381,-0.408541,0.106155,-0.386507


### Baseline event

In [19]:
from timeshap.utils import calc_avg_sequence
average_sequence = calc_avg_sequence(d_train_normalized, numerical_feats=model_features, categorical_feats=[],model_features=model_features, entity_col=sequence_id_feat)

In [20]:
average_sequence

array([[0.0000e+00, 3.9670e+01, 5.0000e-01, 1.4500e+01, 1.2500e+00,
        1.4750e+01],
       [2.5000e+02, 4.0000e+01, 8.2000e-01, 1.4000e+01, 1.0000e+00,
        1.4500e+01],
       [5.0000e+02, 4.0000e+01, 5.0000e-01, 1.4000e+01, 8.7000e-01,
        1.6000e+01],
       ...,
       [1.1925e+05, 4.0250e+01, 8.2000e-01, 1.5330e+01, 9.4000e-01,
        1.5000e+01],
       [1.1950e+05, 3.9500e+01, 5.0000e-01, 1.3750e+01, 1.4100e+00,
        1.5250e+01],
       [1.1975e+05, 3.9750e+01, 5.0000e-01, 1.4000e+01, 9.4000e-01,
        1.4500e+01]])

### Average score over baseline

In [21]:
from timeshap.utils import get_avg_score_with_avg_event
avg_score_over_len = get_avg_score_with_avg_event(f, average_event, top=480)

1/1 [==============================] - 0s 49ms/step


## 3.1 Local Explanations

### Select sequences to explain

In [22]:
positive_sequence_id = f"cycling_{np.random.choice(ids_for_test)}"
pos_x_pd = d_test_normalized[d_test_normalized['all_id'] == positive_sequence_id]

# select model features only
pos_x_data = pos_x_pd[model_features]
# convert the instance to numpy so TimeSHAP receives it
pos_x_data = np.expand_dims(pos_x_data.to_numpy().copy(), axis=0)

### Local Report on positive instance

In [23]:
from timeshap.explainer import local_report

pruning_dict = {'tol': 0.025}
event_dict = {'rs': 42, 'nsamples': 32000}
feature_dict = {'rs': 42, 'nsamples': 32000, 'feature_names': model_features, 'plot_features': plot_feats}
cell_dict = {'rs': 42, 'nsamples': 32000, 'top_x_feats': 2, 'top_x_events': 2}
local_report(f, pos_x_data, pruning_dict, event_dict, feature_dict, cell_dict=cell_dict, entity_uuid=positive_sequence_id, entity_col='all_id', baseline=average_event)

Assuming all features are model features
32/32 [==============================] - 3s 82ms/step


alt.HConcatChart(...)

## 3.2 Global Explanations

### Explain all 

TimeSHAP offers methods to explain all instances and save as CSV.
This allows for global explanations and local plots with no calculation delay.

In [26]:
from timeshap.explainer import global_report

pos_dataset = d_test_normalized[d_test_normalized['label'] == 1]
schema = schema = list(pos_dataset.columns)
pruning_dict = {'tol': [0.05, 0.075], 'path': 'outputs/prun_all_tf.csv'}
event_dict = {'path': 'outputs/event_all_tf.csv', 'rs': 42, 'nsamples': 32000}
feature_dict = {'path': 'outputs/feature_all_tf.csv', 'rs': 42, 'nsamples': 32000, 'feature_names': model_features, 'plot_features': plot_feats}
prun_stats, global_plot = global_report(f, pos_dataset, pruning_dict, event_dict, feature_dict, average_event, model_features, schema, sequence_id_feat, time_feat)
prun_stats

The defined path for pruning data already exists and the append option is turned off. TimeSHAP will only read from this file and will not create new explanation data
The defined path for event explanations already exists and the append option is turned off. TimeSHAP will only read from this file and will not create new explanation data
The defined path for feature explanations already exists and the append option is turned off. TimeSHAP will only read from this file and will not create new explanation data
Calculating pruning algorithm
Calculating event data
Calculating feat data
Calculating pruning indexes


TypeError: Could not convert cycling_1cycling_4cycling_6cycling_8 to numeric

In [25]:
global_plot

NameError: name 'global_plot' is not defined

## 3.3 Individual Plots

### Local Plots

In [27]:
from timeshap.plot import plot_temp_coalition_pruning, plot_event_heatmap, plot_feat_barplot, plot_cell_level
from timeshap.explainer import local_pruning, local_event, local_feat, local_cell_level
# select model features only
pos_x_data = pos_x_pd[model_features]
# convert the instance to numpy so TimeSHAP receives it
pos_x_data = np.expand_dims(pos_x_data.to_numpy().copy(), axis=0)

##### Pruning algorithm

In [28]:
pruning_dict = {'tol': 0.025,}
coal_plot_data, coal_prun_idx = local_pruning(f, pos_x_data, pruning_dict, average_event, positive_sequence_id, sequence_id_feat, False)
# coal_prun_idx is in negative terms
pruning_idx = pos_x_data.shape[1] + coal_prun_idx
pruning_plot = plot_temp_coalition_pruning(coal_plot_data, coal_prun_idx, plot_limit=40)
pruning_plot

1/1 [==============================] - 0s 57ms/step


alt.LayerChart(...)

##### Event-level explanation

In [29]:
event_dict = {'rs': 42, 'nsamples': 32000}
event_data = local_event(f, pos_x_data, event_dict, positive_sequence_id, sequence_id_feat, average_event, pruning_idx)
event_plot = plot_event_heatmap(event_data)
event_plot

1000/1000 [==============================] - 87s 87ms/step


alt.LayerChart(...)

In [30]:
event_plot

alt.LayerChart(...)

##### Feature-level explanation

In [31]:
feature_dict = {'rs': 42, 'nsamples': 32000, 'feature_names': model_features, 'plot_features': plot_feats}
feature_data = local_feat(f, pos_x_data, feature_dict, positive_sequence_id, sequence_id_feat, average_event, pruning_idx)
feature_plot = plot_feat_barplot(feature_data, feature_dict.get('top_feats'), feature_dict.get('plot_features'))
feature_plot

4/4 [==============================] - 0s 85ms/step


alt.LayerChart(...)

##### Cell-level explanation

In [32]:
cell_dict = {'rs': 42, 'nsamples': 32000, 'top_x_events': 3, 'top_x_feats': 3}
cell_data = local_cell_level(f, pos_x_data, cell_dict, event_data, feature_data, positive_sequence_id, sequence_id_feat, average_event, pruning_idx)
feat_names = list(feature_data['Feature'].values)[:-1] # exclude pruned events
cell_plot = plot_cell_level(cell_data, feat_names, feature_dict.get('plot_features'))
cell_plot

1000/1000 [==============================] - 89s 89ms/step


alt.HConcatChart(...)

### Global Plots

In [33]:
from timeshap.explainer import prune_all, pruning_statistics, event_explain_all, feat_explain_all
from timeshap.plot import plot_global_event, plot_global_feat

pos_dataset = d_test_normalized[d_test_normalized['label'] == 1]

##### Pruning statistics

In [34]:
pruning_dict = {'tol': [0.05, 0.075], 'path': 'outputs/prun_all_tf.csv'}
prun_indexes = prune_all(f, pos_dataset, pruning_dict, average_event, model_features, schema, sequence_id_feat, time_feat)
pruning_stats = pruning_statistics(prun_indexes, pruning_dict.get('tol'))
pruning_stats

,Tolerance,Mean,Std
0,0.05,30.75,8.655441
1,0.075,29.00,7.745967
2,No Pruning,480.00,0.000000


##### Global event-level

In [35]:
event_dict = {'path': 'outputs/event_all_tf.csv', 'rs': 42, 'nsamples': 32000}
event_data = event_explain_all(f, pos_dataset, event_dict, prun_indexes, average_event, model_features, schema, sequence_id_feat, time_feat)
event_global_plot = plot_global_event(event_data)
event_global_plot

alt.VConcatChart(...)

##### Global feature-level

In [36]:
feature_dict = {'path': 'outputs/feature_all_tf.csv', 'rs': 42, 'nsamples': 32000, 'feature_names': model_features, 'plot_features': plot_feats, }
feat_data = feat_explain_all(f, pos_dataset, feature_dict, prun_indexes, average_event, model_features, schema, sequence_id_feat, time_feat)
feat_global_plot = plot_global_feat(feat_data, **feature_dict)
feat_global_plot

TypeError: Could not convert cycling_1cycling_4cycling_6cycling_8 to numeric